In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.applications import VGG16
from keras.preprocessing import image
from keras.preprocessing.text import Tokenizer
from keras.applications.vgg16 import preprocess_input
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_pickle('captions_by_path')
df.head()

caption_order,paths,1,2,3,4,5
0,flickr8k/images\1000268201_693b08cb0e.jpg,a child in a pink dress is climbing up a set o...,a girl going into a wooden building .,a little girl climbing into a wooden playhouse .,a little girl climbing the stairs to her playh...,a little girl in a pink dress going into a woo...
1,flickr8k/images\1001773457_577c3a7d70.jpg,a black dog and a spotted dog are fighting,a black dog and a tri-colored dog playing with...,a black dog and a white dog with brown spots a...,two dogs of different breeds looking at each o...,two dogs on pavement moving toward each other .
2,flickr8k/images\1002674143_1b742ab4b8.jpg,a little girl covered in paint sits in front o...,a little girl is sitting in front of a large p...,a small girl in the grass plays with fingerpai...,there is a girl with pigtails sitting in front...,young girl with pigtails painting outside in t...
3,flickr8k/images\1003163366_44323f5815.jpg,a man lays on a bench while his dog sits by him .,a man lays on the bench to which a white dog i...,a man sleeping on a bench outside with a white...,a shirtless man lies on a park bench with his ...,man laying on bench holding leash of dog sitti...
4,flickr8k/images\1007129816_e794419615.jpg,a man in an orange hat starring at something .,a man wears an orange hat and glasses .,a man with gauges and glasses is wearing a bli...,a man with glasses is wearing a beer can croch...,the man with pierced ears is wearing glasses a...


Preprocessing. I will first concatenate the captions into one column such that they can be tokenized.

In [35]:
df['concated'] = df[[1,2,3,4,5]].apply(lambda row: ' caption '.join(row), axis=1)

In [36]:
print(df['concated'].loc[1])

a black dog and a spotted dog are fighting caption a black dog and a tri-colored dog playing with each other on the road . caption a black dog and a white dog with brown spots are staring at each other in the street . caption two dogs of different breeds looking at each other on the road . caption two dogs on pavement moving toward each other .


In [39]:
X = df['paths']
y = df['concated']

In [40]:
y.head()

0    a child in a pink dress is climbing up a set o...
1    a black dog and a spotted dog are fighting cap...
2    a little girl covered in paint sits in front o...
3    a man lays on a bench while his dog sits by hi...
4    a man in an orange hat starring at something ....
Name: concated, dtype: object

In [51]:
print(X_remainder.shape)
print(y_remainder.shape)

(6472,)
(6472,)


In [52]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(y_remainder)
sequences = tokenizer.texts_to_sequences(y_remainder)

In [53]:
print(sequences[1])

[1, 641, 10, 9, 27, 3060, 6, 124, 313, 1, 119, 1579, 2, 1, 10, 9, 1, 13, 1776, 7, 27, 3060, 6, 124, 2, 1, 1776, 10, 366, 27, 4684, 2, 1, 124, 10, 9, 1, 735, 7, 1, 216, 1776, 2, 4, 641, 10, 9, 4, 13, 1776, 6, 20, 1, 4685, 3060]
